In [1]:
#we want to make our client better and have a convenience method to do the dearch and handling authentication method when it expires

In [2]:
#using the spotify api reference guide. we won't have acess to users profile and playlists. I wish! only search albums and artists 

In [3]:
!pip install requests

In [4]:
import requests
import datetime
from urllib.parse import urlencode

In [5]:
import base64

In [6]:
import configparser
config = configparser.ConfigParser()
config.read('/Users/ruby/Desktop/VS-Code-Basics/credentials.ini') #PATH TO MY config file on my local device

cred = config["SPOTIFY"]
client_id = cred["CLIENT_ID"]
client_secret = cred["CLIENT_ID_SECRET"]

#print(client_id)
#print (client_secret)

In [10]:
 
    #not realistic and neat, cuz exception should happen inside perform auth
    #def get_access_token(self)
        #auth_done= self.perform_auth()
        #if not auth_done:
           # raise Exception('Authentication failed')
        #token= self.access_token
        #expires = self.access_token_expires
        #now = datetime.datetime.now()
        #if expires < now:
         #   self.perform_auth()
           # return self.get_access_token()
        #return token
        


In [16]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        #base_url = 'https://api.spotify.com'
        #endpoint = f'{base_url}/v1/albums/{_id}'
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        #base_url = 'https://api.spotify.com'
        #endpoint = f'{base_url}/v1/artists/{_id}'
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

In [17]:
spotify = SpotifyAPI (client_id, client_secret)

In [18]:
spotify.search('Time', search_type = 'Track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
       'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
       'id': '0TnOYISbd1XYRBk9myaseg',
       'name': 'Pitbull',
       'type': 'artist',
       'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
   

In [19]:
spotify.get_artist ('0TnOYISbd1XYRBk9myaseg')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
 'followers': {'href': None, 'total': 9334593},
 'genres': ['dance pop', 'latin', 'miami hip hop', 'pop', 'pop rap'],
 'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
 'id': '0TnOYISbd1XYRBk9myaseg',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb2dc40ac263ef07c16a95af4e',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051742dc40ac263ef07c16a95af4e',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1782dc40ac263ef07c16a95af4e',
   'width': 160}],
 'name': 'Pitbull',
 'popularity': 82,
 'type': 'artist',
 'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}

In [20]:
spotify.search('Work it', search_type = 'Track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Work+it&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2wIVse2owClT7go1WT98tk'},
       'href': 'https://api.spotify.com/v1/artists/2wIVse2owClT7go1WT98tk',
       'id': '2wIVse2owClT7go1WT98tk',
       'name': 'Missy Elliott',
       'type': 'artist',
       'uri': 'spotify:artist:2wIVse2owClT7go1WT98tk'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      

In [21]:
spotify.search('A lannister always pays his debts', search_type = 'Track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=A+lannister+always+pays+his+debts&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
       'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
       'id': '1hCkSJcXREhrodeIHQdav8',
       'name': 'Ramin Djawadi',
       'type': 'artist',
       'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
    

In [23]:
spotify.get_album ('41zMFsCjcGenYKVJYUXU2n')

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
   'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
   'id': '1hCkSJcXREhrodeIHQdav8',
   'name': 'Ramin Djawadi',
   'type': 'artist',
   'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'J

In [ ]:
#imporove the search feature to include more robust queries